# 리뷰 태그화 및 대표 태그 선정
전시/영화 별 리뷰를 모아 대표하는 태그를 선정한다<br>
블로그 형식의 전시 리뷰는 **(1) 요약 후 형용사 추출 -> 대표 형용사 선정**과 (2)방법을 같이 사용<br>
짧은 리뷰가 많은 영화의 경우는 **(2) 전체 리뷰를 대상으로 형용사 추출 및 빈도수(등 기준)에 따라 대표 형용사 선정**

### 전시

In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import csv
from selenium import webdriver
from tqdm.notebook import tqdm
import nltk
import numpy as np
import konlpy
import re

### 리뷰 태그화 및 형용사 추출

In [42]:
import pandas as pd
watcha_review=pd.read_csv('movie_watcha_review.csv', index_col=0)

In [43]:
watcha_review.head()

,movie_id,user,rate,likes,review,en_review
0,0,최한영,4.5,656.0,뜨지 못한 명작,an outstanding masterpiece
1,0,seulkiki,5.0,575.0,사회라는 거대한 틀에서 표류하고있는 두 사람의 극복기. 정말괜찮은 영화.,a really nice movie about two people drifting ...
2,0,DamianJeonDongHyun,4.0,360.0,정말 정재영의 연기중 손에 꼽을 만한 작품이라고 말해주고 싶다.\r\n실제 사회에서...,I really want to tell you that this is one of ...
3,0,새별님,5.0,311.0,나는 이런 영화가 좋더라..감독은 틀림없이 따뜻한 사람일 것이다,I like movies like this. The director must be ...
4,0,BB,5.0,301.0,이 영화에서 가장 난해한건 바로 포스터ㅋ,The most difficult thing in this movie is the ...


In [44]:
review_tag=pd.DataFrame(columns=['movie_id','snt_review','tags'])

for i in range(670):
    movie=watcha_review.loc[watcha_review['movie_id']==i,:]
    snt=""
    for j in range(movie.index[0],movie.index[0]+len(movie)):
        snt+=movie.loc[j,'en_review']+" "
    temp=pd.Series([i,snt,None],index=['movie_id','snt_review','tags'] )
    review_tag=review_tag.append(temp,ignore_index=True)
review_tag.head()

,movie_id,snt_review,tags
0,0,an outstanding masterpiece a really nice movie...,None
1,1,How many people can understand this movie? If ...,None
2,2,"I was ashamed of the movie's social calling, b...",None
3,3,Here's another well-made Spanish thriller. Eve...,None
4,4,Why can't we make disaster films so plain and ...,None


In [12]:
#한글 형용사 추출
for i in range(670):
    review=review_tag.iloc[i,1]
    okt=konlpy.tag.Okt()
    clean_words=[]
    for word in okt.pos(review,stem=True):
        if word[1] in ['Adjective']:
            clean_words.append(word[0])
    clean_words = remove_value(clean_words)
    review_tag.iloc[i,2]=clean_words

In [62]:
def cleaning(word):
    word=re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',word)
    return word
    
def remove_value(in_list):
    val = ['at','in','all','on','the','i','my',
           'no','todays','other','our','ours','you','your','myself']
    return [cleaning(word).lower() for word in in_list if word not in val]

In [48]:
import nltk
# nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import RegexpTokenizer
retokenize=RegexpTokenizer("[\w]+")

In [63]:
#영어 형용사JJ 추출
for i in tqdm(range(len(review_tag))):
    raw_token=retokenize.tokenize(review_tag.at[i,'snt_review'])#각 영화 별 리뷰 문장을 토큰화함
    raw_token=list(filter(lambda x:len(x)>1,raw_token))#길이가 2 이상인 것만 남김
    raw_token=nltk.pos_tag(review_tag.loc[i,'snt_review'].split())#품사를 부여하고
    raw_token=list(filter(lambda x:x[1]=="JJ",raw_token))#형용사만 남김

    #예외처리를 거쳐 최종 tag를 선정
    review_tag.at[i,'tags']=remove_value([raw_token[i][0] for i in range(len(raw_token))])
review_tag.head()

,movie_id,snt_review,tags
0,0,an outstanding masterpiece a really nice movie...,"[outstanding, nice, huge, fresh, real, warm, d..."
1,1,How many people can understand this movie? If ...,"[many, unique, ugly, similar, slow, slow, up, ..."
2,2,"I was ashamed of the movie's social calling, b...","[movies, social, first, current, second, diffe..."
3,3,Here's another well-made Spanish thriller. Eve...,"[wellmade, spanish, spanish, little, fresh, go..."
4,4,Why can't we make disaster films so plain and ...,"[disaster, plain, wellmade, simple, overwhelmi..."


In [64]:
review_tag.to_csv('review_tag.csv')

### convert tag to vector

In [65]:
import pandas as pd
review_tag=pd.read_csv('review_tag.csv', index_col=0)
review_tag.head()

,movie_id,snt_review,tags
0,0,an outstanding masterpiece a really nice movie...,"['outstanding', 'nice', 'huge', 'fresh', 'real..."
1,1,How many people can understand this movie? If ...,"['many', 'unique', 'ugly', 'similar', 'slow', ..."
2,2,"I was ashamed of the movie's social calling, b...","['movies', 'social', 'first', 'current', 'seco..."
3,3,Here's another well-made Spanish thriller. Eve...,"['wellmade', 'spanish', 'spanish', 'little', '..."
4,4,Why can't we make disaster films so plain and ...,"['disaster', 'plain', 'wellmade', 'simple', 'o..."


In [66]:
#all
token=[]
for i in range(670):
    token.extend(review_tag.iloc[i,2].replace('\'','').replace("[",'').replace("]",'').replace(" ","").split(","))

In [67]:
#tags of each movie
token_list=[]
for i in range(len(review_tag)):
    temp=review_tag.iloc[i,2].replace('\'','').replace("[",'').replace("]",'').replace(" ","").split(",")
    token_list.append(temp)

In [68]:
token_dict={}
for word in token:
    try:
        token_dict[word]+=1
    except:
        token_dict[word]=1
token_dict_sorted=sorted(token_dict.items(), key=(lambda x:x[1]), reverse=True)
token_dict_sorted[:5]

[('good', 2595), ('first', 1230), ('its', 1114), ('much', 954), ('many', 953)]

### clustering vectors

In [69]:
import gensim
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
ft_model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

In [70]:
embedding_model = Word2Vec(token_list, size=300, window = 2, min_count=20, workers=4, iter=100, sg=1)

word_vector = embedding_model.wv
vocabs = word_vector.vocab.keys()

word_vectors_list = [word_vector[v] for v in vocabs]

In [71]:
word_vec=pd.DataFrame()
voca_list = list(vocabs)
for i in range(len(word_vectors_list)):
    word_vec[voca_list[i]] = word_vectors_list[i]
word_vec.head()

,outstanding,nice,huge,fresh,real,warm,difficult,only,typical,forced,...,legendary,half,paternal,sensual,cant,thick,koreanstyle,rural,oneman,objective
0,0.215408,-0.140820,0.124796,0.154436,0.061252,0.394711,0.185669,-0.082403,0.307171,-0.585535,...,-0.082666,0.705841,0.173432,0.512547,-0.219057,0.045905,-0.733278,0.034397,-0.276551,0.135116
1,0.050303,-0.124919,0.466957,0.098278,0.211606,-0.303703,0.298860,0.104069,0.311106,-0.032102,...,0.129174,0.709568,-0.176876,-0.030376,0.297140,0.524971,-0.113481,0.295009,0.478547,0.392358
2,-0.592280,-0.069424,-0.039840,-0.086537,-0.213598,-0.204280,-0.187990,-0.241168,0.253978,-0.710035,...,-0.552938,-0.162099,0.013009,-0.588327,0.236518,-0.204476,-0.171389,-0.094971,-0.335389,-0.291845
3,-0.612465,0.364926,-0.106516,-0.451025,-0.176854,-0.469029,-0.161889,0.060869,-0.127780,-0.310237,...,-0.027021,-0.479365,-0.708870,-0.209198,0.310598,-0.141467,0.000011,-0.072944,0.101964,-0.460104
4,0.114133,0.548912,0.037799,0.021293,0.022432,-0.096448,-0.569578,-0.241159,0.115996,-0.578452,...,0.469192,0.071012,-0.059492,0.127089,-0.463619,0.208479,-0.153073,-0.486253,-0.241517,0.455523


In [72]:
k=100

kmeans = KMeans(k)
idx = kmeans.fit_predict(word_vectors_list)

idx = list(idx)
names = embedding_model.wv.index2word
word_centroid_map = {names[i]: idx[i] for i in range(len(names))}

In [73]:
word_dict={}
for i in range(k):
    word_dict[i]=[]
    
for key,value in word_centroid_map.items():
    word_dict[value].append(key)

word_df=pd.DataFrame()
for i in range(k):
    temp=pd.DataFrame({i:word_dict[i]})
    word_df=pd.concat([word_df,temp], ignore_index=True, axis=1)
    
word_df=word_df.fillna("")
word_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,spiderman,dirty,much,know,sentimental,sudden,later,charming,alltime,koreanstyle,...,scared,central,necessary,life,religious,limited,male,heartbreaking,fourth,middle
1,,,many,,,,,,,,...,,,,,,,,,,
2,,,main,,,,,,,,...,,,,,,,,,,
3,,,great,,,,,,,,...,,,,,,,,,,
4,,,last,,,,,,,,...,,,,,,,,,,


In [141]:
word_df.to_csv("adj_cluster.csv")

### maek item-adjective matrix

In [129]:
tag_matrix = pd.DataFrame()
for i in range(len(review_tag)):
    matrix_list=[0]*k
    for tag in token_list[i]:
        try:
            matrix_list[word_centroid_map[tag]]+=1
        except:
            pass
    temp=pd.Series(matrix_list, index=list(range(k)))
    tag_matrix=tag_matrix.append(temp, ignore_index=True)
tag_matrix

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,41.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,67.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,49.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,0.0,0.0,41.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
666,0.0,0.0,22.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
667,0.0,0.0,47.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [130]:
tag_matrix.to_csv("movie_adj_matrix.csv")

### make user-adjective matrix

In [86]:
user_matrix=pd.read_csv('user_matrix.csv', index_col=0)
user_matrix.head()

,movie_id,user_id,rate,en_review
0,0,6151,4.5,an outstanding masterpiece
1,0,5468,5.0,a really nice movie about two people drifting ...
2,0,2160,4.0,I really want to tell you that this is one of ...
3,0,4076,5.0,I like movies like this. The director must be ...
4,0,5531,5.0,The most difficult thing in this movie is the ...


In [101]:
user_list=set(user_matrix['user_id'])

In [126]:
user_review=pd.DataFrame()
for user in user_list:
    temp=""
    reviews=list(user_matrix.loc[user_matrix['user_id']==user,:]['en_review'])
    
    for i in range(len(reviews)):
        temp+=reviews[i]+" "
    
    temp=pd.Series([user,temp,None], index=['user','review','tags'])
    user_review=user_review.append(temp, ignore_index=True)
user_review.head()

,review,tags,user
0,"If you walk out that door, you're the Avengers.",None,0.0
1,Choi Min-sik's charisma dominates the entire m...,None,1.0
2,My cancer got better after watching this movie.,None,2.0
3,Jack Efronb of leather jacket on AudiR8 This i...,None,3.0
4,The movie itself was fascinating.It's similar ...,None,4.0


In [134]:
#토큰화부터 다시 수행하여 유저 별 태그 생성

#영어 형용사JJ 추출
for i in tqdm(range(len(user_review))):
    raw_token=retokenize.tokenize(user_review.at[i,'review'])#각 영화 별 리뷰 문장을 토큰화함
    raw_token=list(filter(lambda x:len(x)>1,raw_token))#길이가 2 이상인 것만 남김
    raw_token=nltk.pos_tag(user_review.loc[i,'review'].split())#품사를 부여하고
    raw_token=list(filter(lambda x:x[1]=="JJ",raw_token))#형용사만 남김

    #예외처리를 거쳐 최종 tag를 선정
    user_review.at[i,'tags']=remove_value([raw_token[i][0] for i in range(len(raw_token))])

#tags of each user
token_list=[]
for i in range(len(user_review)):
    token_list.append(user_review.at[i,'tags'])

In [137]:
user_review.head()

,review,tags,user
0,"If you walk out that door, you're the Avengers.",[],0.0
1,Choi Min-sik's charisma dominates the entire m...,"[entire, first, downtown]",1.0
2,My cancer got better after watching this movie.,[],2.0
3,Jack Efronb of leather jacket on AudiR8 This i...,"[interesting, small, boring, memorable]",3.0
4,The movie itself was fascinating.It's similar ...,"[similar, secret, its, better, happy, actor, s...",4.0


In [139]:
user_tag_matrix = pd.DataFrame()
for i in range(len(user_review)):
    matrix_list=[0]*k
    for tag in token_list[i]:
        try:
            matrix_list[word_centroid_map[tag]]+=1
        except:
            pass
    temp=pd.Series(matrix_list, index=list(range(k)))
    user_tag_matrix=user_tag_matrix.append(temp, ignore_index=True)
user_tag_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [140]:
user_tag_matrix.to_csv('user_adj_matrix.csv')